In [ ]:
# %pip install networkx

In [ ]:
# Base imports
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
# Classifier imports
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Graph of Thrones

Let's take a look at a dataset containing the characters that appear in G.R.R Martin's epic fantasy series *A Song of Ice and Fire*. Can we predict who lives and dies in the first book based on a character's Allegiance, Gender, Nobility and the chapter in which they are introduced? 

In [ ]:
# Get a list of characters from the first book, print the first 5
characters =[]
with open('data/asoiaf-book1-nodes.csv') as f: [characters.append(line.split(',')[1].strip()) for line in f]
characters.pop(0) # pop out the label
characters[:5]

In [ ]:
# Read in data regarding deaths
death_df = pd.read_csv('data/character-deaths.csv')
death_df.head()

In [ ]:
# Create target variable
death_df['Dead'] = death_df['Death Year'].apply(lambda x: 1 if x==x else 0)

In [ ]:
# Compare count of Characters in each set
print(f"""Total Characters: {len(death_df)}, First Book Characters: {len(death_df.query(f'Name in {characters}'))}""")

In [ ]:
# Grab death data from Characters in the first book
first_deaths = death_df.query(f'Name in {characters}').reset_index(drop=True)
first_deaths.head()

In [ ]:
# Grab relevant features and one hot encode categorical data (Allegiances)
features = ['Name','Book Intro Chapter','Gender','Nobility','Dead']
df=pd.concat([first_deaths[features],pd.get_dummies(first_deaths['Allegiances'], prefix='allegiance',drop_first=True)],axis=1)
df.head()


In [ ]:
# Explore the balance of our target class
plt.bar(['Not Dead','Dead'],df.Dead.value_counts());

## Predicting Death

### Tabular Data
>*Valar Morghulis* (All Men must die) -- A Clash of Kings

In [ ]:
# Try a collection of classifiers
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Random Forest", "Neural Net","Logistic Regression"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=3,random_state=42),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    LogisticRegression(solver='liblinear', random_state=42)]

In [ ]:
# Prepare data
X, y = df.drop(['Name','Dead'],axis=1), df['Dead']
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Untuned classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    print(f'Base F1 Score for {name}: {f1_score(y_test, clf.predict(X_test)):.2f}')

We now have some baseline scores for models built solely on tabular data.

### Graph Data
>Every Man must die, but first He must live. -- Storm of Swords

Let's see if we can capture more information about these characters based on their interactions with each other.

In [ ]:
# Import graph database
book1_df = pd.read_csv('data/asoiaf-book1-edges.csv')
G1 = nx.Graph()
for row in book1_df.iterrows():
    G1.add_edge(row[1]['Source'].replace('-',' '), row[1]['Target'].replace('-',' '), weight=row[1]['weight'])

#### EDA
What information do the edge weights capture?

In [ ]:
# Inspect relationship/edge
G1.get_edge_data('Tyrion Lannister','Jon Snow')

We have created a network with characters as **Nodes** connected by **Weighted Edges** where the weight corresponds to the amount of dialogue exchanged with each other in the first book. Is this directed or undirected?

In [ ]:
# Visualize a sample of the graph
plt.figure(figsize=(12,12))
nx.draw(G1.subgraph(df.sample(n=50, random_state=42).Name), with_labels=True, font_weight='bold', pos=nx.spring_layout(G1))
plt.draw()

What does the distribution of weights on the edges look like?

In [ ]:
wts=[]
relationships=[]
# Loop through each node and its adjacent nodes and capture its weight 
for n, nbrs in G1.adj.items():
    for nbr, eattr in nbrs.items():
        wt = eattr['weight']
        wts.append(wt)
        relationships.append((wt,n,nbr))

In [ ]:
# Examine weighted relationship, skip every other one (undirected relationship)
print(sorted(relationships,key=(lambda x: x[0]),reverse=True)[:10:2])

In [ ]:
# Plot distribution of weights
plt.hist(wts);

### Graph Algorithms

Using Graph Theory, let us create some metrics from our network to augment the tabular data used in the models earlier.

#### Degree (number of connections normalized by number of possible connections)


In [ ]:
# Calculate degree and assign it to each node
for name, degree in nx.degree_centrality(G1).items():
    G1.nodes[name]['degree']=degree

In [ ]:
G1.nodes['Tyrion Lannister']

In [ ]:
G1.nodes['Hosteen Frey']

#### Label propagation (community detection algorithm akin to KNN)

In [ ]:
# Assign community label to each node in the community and print out the communities
communities=[]
for label, community in enumerate(nx.algorithms.community.label_propagation.label_propagation_communities(G1)):
    communities.append(community)
    print(f'Community {label}: {community}')
    for name in community:
        G1.nodes[name]['community'] = label

What do these communities tell us about the underlying data?

#### Centrality

In [ ]:
# Assign betweenness metric
for name, betweenness in nx.betweenness_centrality(G1, weight='weight', seed=42).items():
    G1.nodes[name]['betweenness']=betweenness

In [ ]:
G1.nodes['Tyrion Lannister']

In [ ]:
G1.nodes['Hosteen Frey']

What does the betweenness tell us about these two characters?

In [ ]:
# Assign pagerank
for name, rank in nx.pagerank(G1, weight='weight').items():
    G1.nodes[name]['rank']=rank

In [ ]:
G1.nodes['Tyrion Lannister']

In [ ]:
G1.nodes['Hosteen Frey']

What does the pagerank tell us about these two characters?

## Augmented Data

In [ ]:
# Helper function to augment table data
def graph_of_thrones(name):
    if G1.nodes[name]:
        return [G1.nodes[name]['betweenness'], G1.nodes[name]['community'], G1.nodes[name]['rank'], G1.nodes[name]['degree']]
    return [None, None, None, None]

In [ ]:
# Add metrics to original DataFrame
graph_df = pd.concat([df,pd.DataFrame(df.apply(lambda x: graph_of_thrones(x.Name),axis=1).values.tolist(),columns=['Betweenness','Community','Rank','Degree'])],axis=1)
graph_df.columns

In [ ]:
# Create dummy variables for categorical feature
augmented_df=pd.concat([graph_df.drop('Community',axis=1),pd.get_dummies(graph_df['Community'], prefix='community',drop_first=True)],axis=1)

In [ ]:
# Inspect remaining columns
augmented_df.describe()

In [ ]:
# Prepare data 
X, y = augmented_df.drop(['Name','Dead'],axis=1), augmented_df['Dead']
# X = StandardScaler().fit_transform(X)
GX_train, GX_test, Gy_train, Gy_test = \
        train_test_split(X, y, test_size=.2, random_state=42)
g_scaler = StandardScaler()
GX_train = g_scaler.fit_transform(GX_train)
GX_test = g_scaler.transform(GX_test)

In [ ]:
# Untuned Classifiers with number of interactions as weight
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    print(f'Base F1 Score for {name}: {f1_score(y_test, clf.predict(X_test))}')
    clf.fit(GX_train, Gy_train)
    print(f'Graph Augmented F1 Score for {name}: {f1_score(Gy_test, clf.predict(GX_test))} \n')

### Reflection
We can see an improvement in some of the models but a drop in others. Consider how we set up our network with weighted connections; each edge had the number of interactions between the two characters. How do the models think of the weights for their algorithms?

In [ ]:
# The above process has been automated into a pipeline to compare results
from helpers import pipeline

In [ ]:
# Untuned Classifiers with number of interactions as weights
pipeline(df, names, classifiers, 'weight')

In [ ]:
# Untuned Classifiers with no weights
pipeline(df, names, classifiers)

In [ ]:
# Untuned Classifiers with reciprocal number of interactions as weight
pipeline(df, names, classifiers, weight='inverse')

## Conclusion
Graph networks and algorithms are a powerful tool to explore the relationships between objects in our data. Networks can be used not only to visualize data but also to create new features that can be useful for machine learning. It is not the solution to every problem, and there is no guarantee that every class of model will be improved by including this type of data. Like any other type of data it is important to explore your features and examine your assumptions.
> It is one thing to be **clever** and another to be **wise**. -- A Game of Thrones